In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

#from sklearn import datasets
from torchvision import datasets
from sklearn.model_selection import ParameterSampler, RandomizedSearchCV, cross_val_score
from sklearn import metrics
import GPyOpt
import GPy

In [18]:
def load_MNIST():

    # we want to flat the examples

    training_set = datasets.MNIST(root='./data', train=True, download=True, transform= None)
    test_set = datasets.MNIST(root='./data', train=False, download=True, transform= None)

    x_train = training_set.data.numpy().reshape(-1,28*28)
    x_test = test_set.data.numpy().reshape(-1,28*28)

    y_train = training_set.targets.numpy()
    y_test = test_set.targets.numpy()

    return x_train, y_train, x_test, y_test

## we can load the training set and test set
x_train, y_train, x_test, y_test = load_MNIST()

## we use a mask to selects those subsets
train_filter = np.isin(y_train, [3, 5])
test_filter = np.isin(y_test, [3, 5])

# apply the mask to the entire dataset
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]


In [24]:
?GPy.kern.RBF

In [ ]:
np.random.seed(42)

n_neighbors = tuple(np.arange(1,5,1, dtype=np.int))

domain = [
          {'name': 'n_neighbors', 'type': 'discrete', 'domain': n_neighbors},
         ]

def objective_function(x):
    scores_list = []
    param=x[0][0]
    
    model = KNeighborsClassifier(
        n_neighbors = int(param), 
        n_jobs=-1)
    
    model.fit(x_train, y_train)
    score = model.score(x_test, y_test)
    return score

kernel = GPy.kern.RBF(1, variance=1, lengthscale=5)
acquisitions = ['MPI','EI','LCB']
for aq in acquisitions:
    opt = GPyOpt.methods.BayesianOptimization(f = objective_function,   # function to optimize
                                                  domain = domain,         # box-constrains of the problem
                                                  acquisition_type = aq ,      # Select acquisition function MPI, EI, LCB
                                                  kernel=kernel)
    opt.acquisition.exploration_weight=0.5
    opt.run_optimization(max_iter=5) 

    x_best = opt.X[np.argmax(opt.Y)] # argmin -> argmax
    print()
    print("n_neighbors=" + str(x_best[0]))
    print(opt.Y)
    print(opt.X)


n_neighbors=3.0
[[0.98422713]
 [0.98370137]
 [0.98002103]
 [0.98422713]
 [0.98422713]]
[[3.]
 [4.]
 [1.]
 [3.]
 [3.]]


In [21]:
model = KNeighborsClassifier(
        n_neighbors = 3, 
        n_jobs=-1)
model.fit(x_train, y_train)
print(model.score(x_test, y_test))

0.9842271293375394
